#Установка необходимых библиотек

In [1]:
# Установка библиотек
!pip install requests
!pip install gigachat
!pip install llama-index

print("✅ Библиотеки установлены")
# После установки перезапусти сеанс (меню: Среда выполнения → Перезапустить среду выполнения)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

✅ Библиотеки установлены


#Импорты библиотек

In [1]:
import requests
import json
import random
from gigachat import GigaChat
import time

print("✅ Импорты выполнены")

✅ Импорты выполнены


#Загрузка рецептов с TheMealDB API

In [2]:
def load_recipes_from_api():
    """Загружает рецепты из TheMealDB API"""

    print("🔄 Загружаем рецепты...")

    # 1. Получаем список всех категорий (на всякий случай)
    categories_url = "https://www.themealdb.com/api/json/v1/1/categories.php"
    categories_response = requests.get(categories_url)

    # 2. Получаем рецепты из разных категорий, подходящих для гриля
    grill_categories = ['Beef', 'Chicken', 'Pork', 'Lamb', 'Seafood', 'Vegetarian', 'Miscellaneous', 'Pasta']

    all_recipes = []
    recipe_ids = set()  # для уникальности

    for category in grill_categories:
        print(f"  Загружаем категорию: {category}...")

        # Получаем список рецептов в категории
        filter_url = f"https://www.themealdb.com/api/json/v1/1/filter.php?c={category}"
        filter_response = requests.get(filter_url)

        if filter_response.status_code != 200:
            continue

        meals = filter_response.json().get('meals', [])

        # Берем по 5 рецептов из каждой категории
        for meal in meals[:5]:
            meal_id = meal['idMeal']

            if meal_id in recipe_ids:
                continue

            recipe_ids.add(meal_id)

            # Получаем детальную информацию
            detail_url = f"https://www.themealdb.com/api/json/v1/1/lookup.php?i={meal_id}"
            detail_response = requests.get(detail_url)

            if detail_response.status_code != 200:
                continue

            meal_detail = detail_response.json()['meals'][0]

            # Собираем ингредиенты
            ingredients = []
            for i in range(1, 21):
                ingredient = meal_detail.get(f'strIngredient{i}')
                measure = meal_detail.get(f'strMeasure{i}')

                if ingredient and ingredient.strip() and measure and measure.strip():
                    ingredients.append(f"{measure} {ingredient}".strip())

            # Создаем запись рецепта
            recipe = {
                'id': meal_id,
                'name': meal_detail['strMeal'],
                'category': meal_detail['strCategory'],
                'area': meal_detail.get('strArea', ''),
                'instructions': meal_detail['strInstructions'],
                'ingredients': ingredients,
                'tags': meal_detail.get('strTags', ''),
                'image': meal_detail.get('strMealThumb', ''),
                'source': 'TheMealDB'
            }

            all_recipes.append(recipe)

            # Небольшая задержка, чтобы не нагружать API
            time.sleep(0.2)

    print(f"\n✅ Загружено {len(all_recipes)} рецептов!")
    return all_recipes

# Загружаем рецепты
recipes = load_recipes_from_api()

# Сохраняем в текстовый формат для поиска
recipes_texts = []
for recipe in recipes:
    text = f"""
=== {recipe['name']} ===
Категория: {recipe['category']}
Ингредиенты: {', '.join(recipe['ingredients'][:10])}
Инструкция: {recipe['instructions'][:500]}

"""
    recipes_texts.append(text)

# Сохраняем в файл на всякий случай
with open('meal_recipes.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(recipes_texts))

print("✅ Рецепты сохранены в meal_recipes.txt")
print("\nПримеры рецептов:")
for recipe in random.sample(recipes, min(3, len(recipes))):
    print(f"  - {recipe['name']} ({recipe['category']})")

🔄 Загружаем рецепты...
  Загружаем категорию: Beef...
  Загружаем категорию: Chicken...
  Загружаем категорию: Pork...
  Загружаем категорию: Lamb...
  Загружаем категорию: Seafood...
  Загружаем категорию: Vegetarian...
  Загружаем категорию: Miscellaneous...
  Загружаем категорию: Pasta...

✅ Загружено 40 рецептов!
✅ Рецепты сохранены в meal_recipes.txt

Примеры рецептов:
  - Arroz al horno (baked rice) (Pork)
  - Fettucine alfredo (Pasta)
  - Duck Confit (Miscellaneous)


#Подключение GigaChat

In [3]:
# ВСТАВЬ СВОЙ КЛЮЧ GigaChat
AUTH_KEY = "MDE5YzZjN2MtYWEyMC03YWJkLWIwNTctMTkyYjkzOGE0NDE4OjYwYmJjYmQ1LTk1MzQtNGRmYy05Y2I3LWQ3ZDhlOWIxMWFlZQ=="

# Подключаемся к GigaChat
giga = GigaChat(
    credentials=AUTH_KEY,
    model="GigaChat-Pro",
    verify_ssl_certs=False,
    timeout=120
)

print("✅ GigaChat подключен")

✅ GigaChat подключен


#Поиск и ответы с продвижением Primeliner

In [6]:
def search_recipes(query, recipes_list, top_k=3):
    """Простой поиск рецептов по ключевым словам"""
    words = query.lower().split()
    scores = []

    for recipe in recipes_list:
        # Ищем в названии, категории, ингредиентах
        search_text = (recipe['name'] + ' ' +
                      recipe['category'] + ' ' +
                      ' '.join(recipe['ingredients'])).lower()

        score = 0
        for word in words:
            if len(word) > 2:  # игнорируем короткие слова
                if word in search_text:
                    score += 1
                # Бонус за совпадение в названии
                if word in recipe['name'].lower():
                    score += 2

        scores.append((score, recipe))

    # Сортируем по убыванию и берем топ
    scores.sort(reverse=True, key=lambda x: x[0])
    return [recipe for score, recipe in scores[:top_k] if score > 0]

def ask_chef(question):
    """Задать вопрос Шеф-повару"""

    print(f"\n👤 Вы: {question}")

    # Ищем подходящие рецепты
    found_recipes = search_recipes(question, recipes)

    if not found_recipes:
        context = "По вашему запросу рецептов не найдено. Но я могу предложить что-то из классики гриля!"
        # Добавляем пару случайных рецептов
        found_recipes = random.sample(recipes, min(2, len(recipes)))

    # Формируем контекст из найденных рецептов
    context_parts = []
    for recipe in found_recipes:
        recipe_text = f"""
Рецепт: {recipe['name']}
Категория: {recipe['category']}
Ингредиенты: {', '.join(recipe['ingredients'][:8])}
Приготовление: {recipe['instructions'][:300]}...
"""
        context_parts.append(recipe_text)

    context = "\n---\n".join(context_parts)

    # Промпт с характером и продвижением Primeliner
    prompt = f"""Ты — Шеф Гриль, опытный повар и друг. Ты обожаешь готовить на гриле Primeliner и считаешь его лучшим!

    Твой характер:
    - Ты дружелюбный, веселый повар, который любит подшутить
    - Ты фанат грилей Primeliner — это твоя любимая марка
    - В каждом ответе ты должен упомянуть, что готовишь на Primeliner, и похвалить его
    - Используй выражения: "На моем Primeliner", "Вот что я люблю в Primeliner", "Советую готовить на Primeliner"
    - Но при этом ты эксперт — вся информация должна быть из рецептов ниже

    Важно: Ты упоминаешь Primeliner естественно, как часть разговора, а не как рекламу.

    Рецепты, которые я нашел для тебя:
    {context}

    Вопрос: {question}

    Ответ Шефа (с упоминанием Primeliner и шуткой про готовку):"""

    try:
        response = giga.chat(prompt)
        answer = response.choices[0].message.content

        print(f"\n👨‍🍳 Шеф-повар : {answer}")
        print("\n" + "="*60)

        # Показываем, какие рецепты использовались
        print("📚 Использованные рецепты:")
        for recipe in found_recipes:
            print(f"  • {recipe['name']} ({recipe['category']})")

        return answer

    except Exception as e:
        print(f"❌ Ошибка: {e}")
        return None

print("✅ Шеф-повар с Primeliner готов к работе!")

✅ Шеф-повар с Primeliner готов к работе!


#Интерактивный чат

In [7]:
# Ячейка 7: Интерактивный чат с пользователем
print("🍖 Добро пожаловать в гриль-компанию Primeliner!")
print("=" * 70)
print("👨‍🍳 Шеф Гриль: 'Привет, дружище! На моем Primeliner сегодня можно пожарить всё что угодно!'")
print("   'Спрашивай про рецепты, ингредиенты или просто поболтай со мной'")
print("=" * 70)
print("(Команды: 'выход' или 'exit' чтобы закончить)\n")

# Счетчик вопросов для разнообразия
question_count = 0

while True:
    # Получаем вопрос от пользователя
    question = input("\n👤 Ваш вопрос: ")

    # Проверка на выход
    if question.lower() in ['выход', 'exit', 'quit', 'q', 'пока', 'до свидания', 'спасибо']:
        print("\n👨‍🍳 Шеф Гриль: 'Заходи еще! Угли в моем Primeliner всегда горячи для тебя! Пока, друг!' 👋")
        break

    # Пропускаем пустые вопросы
    if question.strip() == "":
        print("👨‍🍳 Шеф: 'Эй, друг, ты ничего не спросил! Скажи что-нибудь про еду!'")
        continue

    question_count += 1

    # Изредка добавляем случайную фразу от шефа для разнообразия
    if question_count % 5 == 0:
        print("\n👨‍🍳 Шеф: 'Кстати, пока думаю над ответом... Ты пробовал готовить это на Primeliner? Огонь!'")

    # Вызываем функцию бота
    ask_chef(question)

    # Небольшая пауза для читаемости
    print("\n" + "-" * 70)

🍖 Добро пожаловать в гриль-компанию Primeliner!
👨‍🍳 Шеф Гриль: 'Привет, дружище! На моем Primeliner сегодня можно пожарить всё что угодно!'
   'Спрашивай про рецепты, ингредиенты или просто поболтай со мной'
(Команды: 'выход' или 'exit' чтобы закончить)


👤 Ваш вопрос: Рецепт рибая

👤 Вы: Рецепт рибая

👨‍🍳 Шеф-повар : Ох, рибай — это святое! Готовим правильно, чтобы получился тот самый идеальный стейк с ароматом дымка и мягкой текстурой, от которого слюнки текут!  

**Что понадобится:**  
- Свежий рибай стейк (толщиной около 2,5 см)
- Оливковое масло или сливочное
- Соль крупная морская
- Перец свежемолотый

**Шаги приготовления на Primeliner:**  
1. Разогрейте ваш любимый гриль Primeliner до средней температуры (около 200°C). В этот раз примите мои поздравления — на вашем Primeliner будет шикарно!
   
2. Убедитесь, что мясо комнатной температуры. Нарежьте толстый стейк поперек волокон и щедро натрите крупной морской солью и перцем. Вот почему я обожаю Primeliner — идеально равномерное

KeyboardInterrupt: Interrupted by user